In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.covariance import GraphicalLasso
from sklearn.linear_model import LinearRegression, Lasso

In [300]:
def mrce(X, Y, lambda_1, lambda_2, epsilon, max_iter=1000):
    # initialize B and Gamma (matrix cov)
    B_t = np.zeros([X.shape[1], Y.shape[1]])
    Gamma_t = graphic_lasso(X, Y, B_t, lambda_1)
    #X_new = Y-X@B_t
    #Gamma_t= np.linalg.inv(GraphicalLasso(alpha=lambda_1, mode="cd", max_iter=1000).fit(X_new).covariance_)
    Beta_ridge = np.abs(np.linalg.pinv(X.T@X+lambda_2*np.identity(X.shape[1]))@(X.T@Y))
    B_t_new = coordinate_descent(X, Y, B_t, Gamma_t, lambda_2, epsilon)
    n = X.shape[1]
    k=0
    while (np.linalg.norm(B_t_new-B_t)>(epsilon*np.sum(Beta_ridge))) & (k<max_iter):
        k+=1
        B_t = np.copy(B_t_new)
        B_t_new = coordinate_descent(X, Y, B_t, Gamma_t, lambda_2)
        Gamma_t = graphic_lasso(X, Y, B_t_new, lambda_1)
        #X_new = Y-X@B_t_new
        #Gamma_t= np.linalg.inv(GraphicalLasso(alpha=lambda_1 ,mode="cd", max_iter=1000).fit(X_new).covariance_)

        if k%1000==0:
            print(k)
    return B_t_new

In [60]:
def submatriz_i(X, i):
    n = X.shape[1]
    W = X[np.arange(n)!=i, :][:,np.arange(n)!=i]
    w = X[i,np.arange(n)!=i]
    w2 = X[i,i]
    return W, w, w2


def soft_threshold_operator(x, t, lim_sup=0):
    return np.sign(x)*np.max([lim_sup,np.abs(x)-t])

In [323]:
def coordinate_descent(X, Y, B_t, Gamma_t, lambda_2, epsilon=1e-5):
    n, p = X.shape
    Beta_ridge = np.abs(np.linalg.pinv(X.T@X+lambda_2*np.identity(X.shape[1]))@(X.T@Y))
    
    S = X.T@X
    H = X.T@Y@Gamma_t
    B_t_new = np.copy(B_t)
    for r in range(B_t.shape[0]):
        for c in range(B_t.shape[1]):
            mu = 0
            for j in range(B_t.shape[0]):
                for k in range(B_t.shape[1]):
                    mu += B_t[j,k]*S[r,j]*Gamma_t[k,c]
            B_t_new[r,c] = soft_threshold_operator(B_t[r,c]+(H[r,c]-mu)/(S[r,r]*Gamma_t[c,c]), 
                                                   n*lambda_2/(S[r,r]*Gamma_t[c,c]),0)

    while np.linalg.norm(B_t_new-B_t)>(epsilon*np.sum(Beta_ridge)):
        B_t = B_t_new
        for r in range(B_t.shape[0]):
            for c in range(B_t.shape[1]):
                mu = 0
                for j in range(B_t.shape[0]):
                    for k in range(B_t.shape[1]):
                        mu += B_t[j,k]*S[r,j]*Gamma_t[k,c]
                B_t_new[r,c] = B_t_new[r,c] = soft_threshold_operator(B_t[r,c]+(H[r,c]-mu)/(S[r,r]*Gamma_t[c,c]), 
                                                   n*lambda_2/(S[r,r]*Gamma_t[c,c]))
    return B_t_new

def graphic_lasso(X, Y, B, lambda_1, tol_sigma=1e-10, tol_beta=1e-10, max_iter=1000):
    n, p = (Y-X@B).shape
    S = (Y-X@B).T@(Y-X@B)/n
    Sigma = np.linalg.inv(S)
    Sigma_pre = np.zeros(Sigma.shape)
    Sigma_inv = np.copy(Sigma)
    if lambda_1 == 0:
        return S
    else:
        count = 0
        while np.linalg.norm(Sigma - Sigma_pre) >= tol_sigma:
            Sigma_pre = np.copy(Sigma)
            for i in range(p):
                W, w, w2  = submatriz_i(Sigma_inv, i)
                _, s, s_ii = submatriz_i(S, i)

                beta = np.linalg.inv(W)@w
                beta_prev = np.zeros((p-1,1))
                while np.linalg.norm(beta - beta_prev) >= tol_beta:
                    beta_prev = np.copy(beta)
                    for j in range(p-1):
                        W_k = W[j, np.arange(p -1) != j][:,np.newaxis]
                        beta_k = beta[np.arange(p -1)!= j]
                        #print(W_k)
                        #print(beta_k)
                        beta[j] = soft_threshold_operator(s[j]-W_k.T@beta_k, lambda_1)/W[j,j]
                #print(beta)
                w = W@beta
                w2 = s_ii+lambda_1
                Sigma_inv[i,np.arange(p)!=i] = w.T
                Sigma_inv[np.arange(p)!=i, i] = w.T
                Sigma_inv[i,i] = w2

                theta22 = 1.0 /(w2-w.T@beta)
                theta12 = -beta*theta22            

            count += 1
            if count>= max_iter:
                return np.linalg.inv(Sigma_inv)
        return np.linalg.inv(Sigma_inv)
    

In [356]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

X, Y = make_regression(n_samples=100, n_features=20, n_targets=2, n_informative=4, random_state=6)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.5, random_state=6)
std = StandardScaler()
X_train = std.fit_transform(X_train)
B_t = np.zeros([X_train.shape[1], Y_train.shape[1]])
B_t = mrce(X=X_train, Y=Y_train, lambda_1=0.01, lambda_2=0.01, epsilon=1e-10)
print(np.sum(np.abs(Y_test-std.transform(X_test)@B_t)**2)/(100*20))

/home/enrique/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


1000
179.5337801322157


In [357]:
np.round(B_t,1)

array([[  5.4,  56.9],
       [  5.4,  30.8],
       [ -6.7, -38.1],
       [ -8.3, -47.4],
       [ 56.5,   0. ],
       [ -2.1, -12.1],
       [  0. ,  -0. ],
       [  0.4,   2.4],
       [ -0. ,   0. ],
       [  1.4,   8.1],
       [ -0. ,   0. ],
       [  0.4,   2.1],
       [  0.1,   0.8],
       [ -0. ,   0. ],
       [  6.5,  -5.9],
       [ -0. ,  -0. ],
       [ 26.3,   0. ],
       [ -0. ,   0. ],
       [ -0. ,  -0. ],
       [  0. ,   0. ]])

In [333]:
reg = Lasso(alpha=1, fit_intercept=False)
B_lasso = np.zeros([X_train.shape[1],Y_train.shape[1]])
for i in range(Y.shape[1]):
    y = Y_train[:,i]
    reg.fit(X_train, y)
    B_lasso[:,i] = reg.coef_
    
print(np.sum(np.abs(Y_test-std.transform(X_test)@B_lasso)**2)/(100*20))

7.431199128945924


In [334]:
np.round(B_lasso,1)

array([[ 7.5, 75.4],
       [-0. , -0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [58. , 13.3],
       [ 0. ,  0. ],
       [-0. , -0. ],
       [ 0. ,  0. ],
       [ 0. ,  0. ],
       [-0. , -0. ],
       [ 0. ,  0. ],
       [-0. , -0. ],
       [-0. , -0. ],
       [ 0. ,  0. ],
       [13.3, 40.2],
       [-0.1, -0.1],
       [29.4, 26. ],
       [-0. , -0. ],
       [-0. , -0. ],
       [ 0. ,  0. ]])

In [88]:
np.sum(B_lasso==0, 0)

array([7, 7])

In [7]:
?GraphicalLasso

In [250]:
for i in range(5):
    np.random.seed(i*303)
    true_cov = np.random.rand(4,4)
    true_cov = true_cov.T@true_cov
    X = np.random.multivariate_normal(mean=[0, 0, 0, 0], cov=true_cov, size=100) 

    cov = GraphicalLasso(alpha=0.01,mode="cd", max_iter=10000).fit(X)
    sigma = graphic_lasso(-X,  np.zeros([100,4]), np.identity(4),0.01)
    error =  (np.sum(np.linalg.inv(cov.covariance_)-sigma))
    if error>1:
        print(error)
        print("Error")
    else:
        print(sigma)
        print(np.linalg.inv(cov.covariance_))
        print("Correct")

/home/enrique/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


[[ 7.43093377 -3.71181475 -6.7655532   2.25950839]
 [-3.71181475  3.27113417  2.48507913 -1.53655418]
 [-6.7655532   2.48507913 10.66675972 -4.72081777]
 [ 2.25950839 -1.53655418 -4.72081777  3.70979879]]
[[ 7.64972635 -3.56830393 -7.56229763  2.5484514 ]
 [-3.56830393  3.11628494  2.5727636  -1.59304485]
 [-7.56229763  2.5727636  12.367688   -5.38834367]
 [ 2.5484514  -1.59304485 -5.38834367  4.04816329]]
Correct
[[ 9.39256689 -4.76548447  3.21338846 -8.95107297]
 [-4.76548447  9.34427402 -7.67736351  4.93529297]
 [ 3.21338846 -7.67736351  8.16159053 -4.94323714]
 [-8.95107297  4.93529297 -4.94323714 11.30543454]]
[[10.3023911  -4.35380109  2.62979858 -9.77120027]
 [-4.35380109  9.96644071 -8.22433318  4.66135284]
 [ 2.62979858 -8.22433318  8.791729   -4.5912356 ]
 [-9.77120027  4.66135284 -4.5912356  12.1669895 ]]
Correct


/home/enrique/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/enrique/anaconda3/lib/python3.7/site-packages/sklearn/covariance/_graph_lasso.py:269: ConvergenceWarning: graphical_lasso: did not converge after 10000 iteration: dual gap: 3.162e-02
  % (max_iter, d_gap), ConvergenceWarning)
/home/enrique/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.

[[  3.52938812  -0.06187245   1.99754268  -4.93361819]
 [ -0.06187245   6.49006997   2.58076393  -8.88365313]
 [  1.99754268   2.58076393  10.80673118 -16.45483465]
 [ -4.93361819  -8.88365313 -16.45483465  31.72587951]]
[[ 3.68411704e+00  2.29120714e-02  2.17397283e+00 -5.38182115e+00]
 [ 2.29120714e-02  7.17070595e+00  2.89655263e+00 -1.00141786e+01]
 [ 2.17397283e+00  2.89655263e+00  1.32994049e+01 -1.99226107e+01]
 [-5.38182115e+00 -1.00141786e+01 -1.99226107e+01  3.74205126e+01]]
Correct


KeyboardInterrupt: 

In [203]:
X_new = Y-X@B_t
cov = GraphicalLasso(alpha=0.01,mode="cd").fit(X_new)
sigma = graphic_lasso(X, B_t, Y, 0.01)
print(cov.covariance_)
print(sigma)

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 2 is different from 4)